In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

C:\Anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
test=pd.read_csv('test.csv')

In [3]:
test

,id,date,product_id,price,discount,season_type,on_promotion,promotion_type,store_count
0,1,2021-08-09,1,340.73,0.0000,Autumn-Winter,1,Main Promo 3,260
1,2,2021-08-16,1,340.73,0.0000,Autumn-Winter,1,Main Promo 3,247
2,3,2021-08-23,1,340.73,0.0000,Autumn-Winter,1,Main Promo 3,260
3,4,2021-08-30,1,340.73,0.0000,Autumn-Winter,1,Main Promo 3,260
4,5,2021-09-06,1,340.73,0.0000,Autumn-Winter,1,Main Promo 3,260
...,...,...,...,...,...,...,...,...,...
137635,137636,2022-01-03,7884,131.93,0.0001,Summer-Spring,1,Main Promo 1,2
137636,137637,2022-01-10,7884,131.93,0.0001,Summer-Spring,1,Main Promo 2,1
137637,137638,2022-03-07,7884,131.93,0.0001,Summer-Spring,0,NaN,1
137638,137639,2022-03-14,7884,131.93,0.0001,Summer-Spring,0,NaN,1


# Seasons

In [4]:
seasonlist=[]
for i in test["season_type"]:
    if i == "Autumn-Winter":
        seasonlist.append(-1)
    else:
        seasonlist.append(1)
seasondata=pd.DataFrame(seasonlist,columns=["season"])
seasondata

,season
0,-1
1,-1
2,-1
3,-1
4,-1
...,...
137635,1
137636,1
137637,1
137638,1


# Promotions

In [5]:
check=[]
count=0
for j in test["promotion_type"]:
    if j not in check:
        check.append(j)
        count+=1
print(count)
check

6


['Main Promo 3',
 'Season Middle Promo 1',
 nan,
 'Genaral Promo- No label discount',
 'Main Promo 1',
 'Main Promo 2']

In [6]:
mainpromolist=[]
for k in test["promotion_type"]:
    if k == "Main Promo 3":
        mainpromolist.append(3)
    elif k == "Main Promo 2":
        mainpromolist.append(2)
    elif k == "Main Promo 1":
        mainpromolist.append(1)
    else:
        mainpromolist.append(0)

mainpromodata=pd.DataFrame(mainpromolist,columns=["main_promo"])
mainpromodata


,main_promo
0,3
1,3
2,3
3,3
4,3
...,...
137635,1
137636,2
137637,0
137638,0


In [7]:
seasonalpromolist=[]
for k in test["promotion_type"]:
    if k == "Season Middle Promo 1":
        seasonalpromolist.append(1)
    elif k == "Season Middle Promo 2":
        seasonalpromolist.append(2)
    else:
        seasonalpromolist.append(0)

seasonalpromodata=pd.DataFrame(seasonalpromolist,columns=["seasonal_promo"])
seasonalpromodata


,seasonal_promo
0,0
1,0
2,0
3,0
4,0
...,...
137635,0
137636,0
137637,0
137638,0


In [8]:
generalpromo=[]
for k in test["promotion_type"]:
    if k == "Genaral Promo- No label discount":
        generalpromo.append(1)
    else:
        generalpromo.append(0)

generalpromodata=pd.DataFrame(generalpromo,columns=["general_promo"])
generalpromodata


,general_promo
0,0
1,0
2,0
3,0
4,0
...,...
137635,0
137636,0
137637,0
137638,0


In [9]:
blackfriday=[]
for k in test["promotion_type"]:
    if k == "BlackFriday":
        blackfriday.append(1)
    else:
        blackfriday.append(0)

blackfridaydata=pd.DataFrame(blackfriday,columns=["black_friday"])
blackfridaydata


,black_friday
0,0
1,0
2,0
3,0
4,0
...,...
137635,0
137636,0
137637,0
137638,0


In [10]:
test["season"]=seasondata
test["main_promo"]=mainpromodata
test["seasonal_promo"]=seasonalpromodata
test["general_promo"]=generalpromodata
test["black_friday"]=blackfridaydata
test

,id,date,product_id,price,discount,season_type,on_promotion,promotion_type,store_count,season,main_promo,seasonal_promo,general_promo,black_friday
0,1,2021-08-09,1,340.73,0.0000,Autumn-Winter,1,Main Promo 3,260,-1,3,0,0,0
1,2,2021-08-16,1,340.73,0.0000,Autumn-Winter,1,Main Promo 3,247,-1,3,0,0,0
2,3,2021-08-23,1,340.73,0.0000,Autumn-Winter,1,Main Promo 3,260,-1,3,0,0,0
3,4,2021-08-30,1,340.73,0.0000,Autumn-Winter,1,Main Promo 3,260,-1,3,0,0,0
4,5,2021-09-06,1,340.73,0.0000,Autumn-Winter,1,Main Promo 3,260,-1,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137635,137636,2022-01-03,7884,131.93,0.0001,Summer-Spring,1,Main Promo 1,2,1,1,0,0,0
137636,137637,2022-01-10,7884,131.93,0.0001,Summer-Spring,1,Main Promo 2,1,1,2,0,0,0
137637,137638,2022-03-07,7884,131.93,0.0001,Summer-Spring,0,NaN,1,1,0,0,0,0
137638,137639,2022-03-14,7884,131.93,0.0001,Summer-Spring,0,NaN,1,1,0,0,0,0


In [11]:
testdata=test.drop(columns=["season_type","promotion_type"])
testdata

,id,date,product_id,price,discount,on_promotion,store_count,season,main_promo,seasonal_promo,general_promo,black_friday
0,1,2021-08-09,1,340.73,0.0000,1,260,-1,3,0,0,0
1,2,2021-08-16,1,340.73,0.0000,1,247,-1,3,0,0,0
2,3,2021-08-23,1,340.73,0.0000,1,260,-1,3,0,0,0
3,4,2021-08-30,1,340.73,0.0000,1,260,-1,3,0,0,0
4,5,2021-09-06,1,340.73,0.0000,1,260,-1,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
137635,137636,2022-01-03,7884,131.93,0.0001,1,2,1,1,0,0,0
137636,137637,2022-01-10,7884,131.93,0.0001,1,1,1,2,0,0,0
137637,137638,2022-03-07,7884,131.93,0.0001,0,1,1,0,0,0,0
137638,137639,2022-03-14,7884,131.93,0.0001,0,1,1,0,0,0,0


# Products

In [12]:
new=pd.read_csv("product.csv")

In [13]:
new2=new.drop(columns="id")
new2=new2.fillna(-1)
new2

,category_1,category_2,category_3,color_type,life_style,fabric,weight_of_fabric,neck_style,form_type,sleeve_type,washing_style,fabric_type
0,Thick Tops,MENSWEAR,Overtees,Green,Daily,versatile,Medium,Bendy,Normal,Long,washless,Polyester
1,Thick Tops,MENSWEAR,Overtees,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,Thick Tops,MENSWEAR,Overtees,Blue,Daily,versatile,Medium,-1,Normal,Long,washless,Polyester
3,Thick Tops,MENSWEAR,Overtees,Black,Daily,versatile,Medium,-1,Normal,Long,washless,Polyester
4,Thick Tops,MENSWEAR,Overtees,Black,Daily Sunday,versatile,Medium,Bendy,Normal,Long,washless,Polyester
...,...,...,...,...,...,...,...,...,...,...,...,...
7871,T-shirts,WOMENSWEAR,Long Sleeves,White,Township,versatile,Medium,High,Fit,Long,washless,Cotton
7872,T-shirts,WOMENSWEAR,Long Sleeves,Black,Visual,Lace,Medium,Bycyle,Very Large,Long,washless,Cotton
7873,T-shirts,WOMENSWEAR,Long Sleeves,Black,Visual,Lace,Medium,Bycyle,Very Large,Long,washless,Cotton
7874,T-shirts,WOMENSWEAR,Long Sleeves,Black,Visual,Lace,Medium,Bycyle,Very Large,Long,washless,Cotton


In [14]:
bos=new["id"]
count=0
for i in new2:
    listnew=[]
    listcat=[]
    
    for j in new2[i]:
        if (j==-1 ) or (j=="Null"):
            listcat.append(np.nan)
            count+=1
        else:
            if j not in listnew:
                listnew.append(j)
            listcat.append(listnew.index(j)+1)
    #print(listcat)
    bos=pd.concat([bos,pd.DataFrame(listcat,columns=[i])],axis=1)
#print(count)
bos



,id,category_1,category_2,category_3,color_type,life_style,fabric,weight_of_fabric,neck_style,form_type,sleeve_type,washing_style,fabric_type
0,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,1,1,2.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0
3,4,1,1,1,3.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0
4,5,1,1,1,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7871,7884,2,2,4,9.0,22.0,1.0,1.0,2.0,2.0,1.0,1.0,4.0
7872,7885,2,2,4,3.0,15.0,7.0,1.0,3.0,4.0,1.0,1.0,4.0
7873,7886,2,2,4,3.0,15.0,7.0,1.0,3.0,4.0,1.0,1.0,4.0
7874,7887,2,2,4,3.0,15.0,7.0,1.0,3.0,4.0,1.0,1.0,4.0


# k-NN for filling the missing info in products

In [15]:
from sklearn.metrics.pairwise import nan_euclidean_distances
from sklearn.impute import KNNImputer

In [16]:
imputer=KNNImputer(n_neighbors=5)
z=imputer.fit_transform(bos)
q=pd.DataFrame(z,columns=["id","category_1","category_2","category_3","color_type","life_style","fabric","weight_of_fabric","neck_style","form_type","sleeve_type","washing_style","fabric_type"])
newdata=q["id"]
for i in q:
    templist=[]
    for j in q[i]:
        templist.append(int(j))
    newdata=pd.concat([newdata,pd.DataFrame(templist,columns=[i])],axis=1)
newdata
    

,id,id,category_1,category_2,category_3,color_type,life_style,fabric,weight_of_fabric,neck_style,form_type,sleeve_type,washing_style,fabric_type
0,1.0,1,1,1,1,1,1,1,1,1,1,1,1,1
1,2.0,2,1,1,1,2,1,1,1,1,1,1,1,1
2,3.0,3,1,1,1,2,1,1,1,1,1,1,1,1
3,4.0,4,1,1,1,3,1,1,1,1,1,1,1,1
4,5.0,5,1,1,1,3,2,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7871,7884.0,7884,2,2,4,9,22,1,1,2,2,1,1,4
7872,7885.0,7885,2,2,4,3,15,7,1,3,4,1,1,4
7873,7886.0,7886,2,2,4,3,15,7,1,3,4,1,1,4
7874,7887.0,7887,2,2,4,3,15,7,1,3,4,1,1,4


In [18]:
bos=newdata.iloc[:,1:]

In [19]:
bos

,id,category_1,category_2,category_3,color_type,life_style,fabric,weight_of_fabric,neck_style,form_type,sleeve_type,washing_style,fabric_type
0,1,1,1,1,1,1,1,1,1,1,1,1,1
1,2,1,1,1,2,1,1,1,1,1,1,1,1
2,3,1,1,1,2,1,1,1,1,1,1,1,1
3,4,1,1,1,3,1,1,1,1,1,1,1,1
4,5,1,1,1,3,2,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7871,7884,2,2,4,9,22,1,1,2,2,1,1,4
7872,7885,2,2,4,3,15,7,1,3,4,1,1,4
7873,7886,2,2,4,3,15,7,1,3,4,1,1,4
7874,7887,2,2,4,3,15,7,1,3,4,1,1,4


In [20]:
denemelist=[]
bosnp=bos.to_numpy()
for i in testdata["product_id"]:
    if i<48:
        denemelist.append(bosnp[i-1])
    elif i>47 and i<333:
        denemelist.append(bosnp[i-2])
    elif i>332 and i<2165:
        denemelist.append(bosnp[i-3])
    elif i>2164 and i<2343:
        denemelist.append(bosnp[i-4])
    elif i>2342 and i<2938:
        denemelist.append(bosnp[i-5])
    elif i>2937 and i<3571:
        denemelist.append(bosnp[i-6])
    elif i>3570 and i<3590:
        denemelist.append(bosnp[i-7])
    elif i>3589 and i<3733:
        denemelist.append(bosnp[i-8])
    elif i>3732 and i<3793:
        denemelist.append(bosnp[i-9])
    elif i>3792 and i<4586:
        denemelist.append(bosnp[i-10])
    elif i>4585 and i<5392:
        denemelist.append(bosnp[i-11])
    elif i>5391 and i<6491:
        denemelist.append(bosnp[i-12])
    else:
        denemelist.append(bosnp[i-13])
len(denemelist)

137640

In [21]:
denpd=pd.DataFrame(denemelist,columns=["id","category_1","category_2","category_3","color_type","life_style","fabric","weight_of_fabric","neck_style","form_type","sleeve_type","washing_style","fabric_type"])

In [22]:
denpd2=denpd.drop(columns="id")
denpd2

,category_1,category_2,category_3,color_type,life_style,fabric,weight_of_fabric,neck_style,form_type,sleeve_type,washing_style,fabric_type
0,1,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
137635,2,2,4,9,22,1,1,2,2,1,1,4
137636,2,2,4,9,22,1,1,2,2,1,1,4
137637,2,2,4,9,22,1,1,2,2,1,1,4
137638,2,2,4,9,22,1,1,2,2,1,1,4


In [23]:
all_test=pd.concat([testdata,denpd2],axis=1)
all_test

,id,date,product_id,price,discount,on_promotion,store_count,season,main_promo,seasonal_promo,...,category_3,color_type,life_style,fabric,weight_of_fabric,neck_style,form_type,sleeve_type,washing_style,fabric_type
0,1,2021-08-09,1,340.73,0.0000,1,260,-1,3,0,...,1,1,1,1,1,1,1,1,1,1
1,2,2021-08-16,1,340.73,0.0000,1,247,-1,3,0,...,1,1,1,1,1,1,1,1,1,1
2,3,2021-08-23,1,340.73,0.0000,1,260,-1,3,0,...,1,1,1,1,1,1,1,1,1,1
3,4,2021-08-30,1,340.73,0.0000,1,260,-1,3,0,...,1,1,1,1,1,1,1,1,1,1
4,5,2021-09-06,1,340.73,0.0000,1,260,-1,3,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137635,137636,2022-01-03,7884,131.93,0.0001,1,2,1,1,0,...,4,9,22,1,1,2,2,1,1,4
137636,137637,2022-01-10,7884,131.93,0.0001,1,1,1,2,0,...,4,9,22,1,1,2,2,1,1,4
137637,137638,2022-03-07,7884,131.93,0.0001,0,1,1,0,0,...,4,9,22,1,1,2,2,1,1,4
137638,137639,2022-03-14,7884,131.93,0.0001,0,1,1,0,0,...,4,9,22,1,1,2,2,1,1,4


In [24]:
import datetime as dt
all_test["date"]=pd.to_datetime(all_test["date"])
all_test['year']=all_test['date'].dt.year
year=all_test['year']
all_test['month']=all_test['date'].dt.month
month=all_test['month']
all_test['day']=all_test['date'].dt.day
day=all_test['day']
all_test["weekofyear"]=all_test["date"].dt.weekofyear
weekofyear=all_test["weekofyear"]

<ipython-input-24-d98ac50cffbb>:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  all_test["weekofyear"]=all_test["date"].dt.weekofyear


In [25]:
testlast=all_test.drop(columns=["date","id"])
testlast

,product_id,price,discount,on_promotion,store_count,season,main_promo,seasonal_promo,general_promo,black_friday,...,weight_of_fabric,neck_style,form_type,sleeve_type,washing_style,fabric_type,year,month,day,weekofyear
0,1,340.73,0.0000,1,260,-1,3,0,0,0,...,1,1,1,1,1,1,2021,8,9,32
1,1,340.73,0.0000,1,247,-1,3,0,0,0,...,1,1,1,1,1,1,2021,8,16,33
2,1,340.73,0.0000,1,260,-1,3,0,0,0,...,1,1,1,1,1,1,2021,8,23,34
3,1,340.73,0.0000,1,260,-1,3,0,0,0,...,1,1,1,1,1,1,2021,8,30,35
4,1,340.73,0.0000,1,260,-1,3,0,0,0,...,1,1,1,1,1,1,2021,9,6,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137635,7884,131.93,0.0001,1,2,1,1,0,0,0,...,1,2,2,1,1,4,2022,1,3,1
137636,7884,131.93,0.0001,1,1,1,2,0,0,0,...,1,2,2,1,1,4,2022,1,10,2
137637,7884,131.93,0.0001,0,1,1,0,0,0,0,...,1,2,2,1,1,4,2022,3,7,10
137638,7884,131.93,0.0001,0,1,1,0,0,0,0,...,1,2,2,1,1,4,2022,3,14,11


# Holidays

In [26]:
holidays=pd.read_csv("holidays.csv")

In [27]:
holidays["date"]=pd.to_datetime(holidays["date"])
holidays['year']=holidays['date'].dt.year
year=holidays['year']
holidays['month']=holidays['date'].dt.month
month=holidays['month']
holidays['day']=holidays['date'].dt.day
day=holidays['day']
holidays["weekofyear"]=holidays["date"].dt.weekofyear
weekofyear=holidays["weekofyear"]

<ipython-input-27-b9113463a89c>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  holidays["weekofyear"]=holidays["date"].dt.weekofyear


In [28]:
allyears=[]
allweeks=[]
for i in testlast["year"]:
    allyears.append(i)
for j in testlast["weekofyear"]:
    allweeks.append(j)
hyears=[]
hweeks=[]
for i in holidays["year"]:
    hyears.append(i)
for j in holidays["weekofyear"]:
    hweeks.append(j)

In [29]:
hlist=[]
hslist=[]
hcount=0
count=0
for i in holidays["holiday"]:
    count+=1
    if i not in hlist:
        hlist.append(i)
        hslist.append(count-1)
        hcount+=1
print(hslist)
#hlist

[0, 3, 8, 14, 20, 26, 61, 81, 82, 117, 152, 187, 222, 257, 292, 297, 299, 302, 307, 308, 313, 322, 327, 337, 342, 347, 362, 377, 392, 407, 437, 450]


In [30]:
bc=0
for i in range(hcount-1):
    templist=[]
    for y in range(len(allyears)):
        tc=0
        for j in range(hslist[i],hslist[i+1]):
            if (hyears[j]==allyears[y] and hweeks[j]==allweeks[y]) or (hyears[j]==allyears[y] and hweeks[j]==allweeks[y]-1):
                tc=1
                if tc>0:
                    break
        templist.append(tc)
    bc+=1
    testlast[hlist[i]]=pd.DataFrame(templist)
    print(bc)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [31]:
testlast

,product_id,price,discount,on_promotion,store_count,season,main_promo,seasonal_promo,general_promo,black_friday,...,StatehoodDay,VisuallyImpairedPeopleDay,WhitMonday,WhitSunday,YouthDay,fathers_day,mothers_day,new_years,new_years_eve,valentines_day
0,1,340.73,0.0000,1,260,-1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,340.73,0.0000,1,247,-1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,340.73,0.0000,1,260,-1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,340.73,0.0000,1,260,-1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,340.73,0.0000,1,260,-1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137635,7884,131.93,0.0001,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137636,7884,131.93,0.0001,1,1,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137637,7884,131.93,0.0001,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137638,7884,131.93,0.0001,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
testlast.to_csv('knntestout.csv')